Solve the 1D Navier-Stokes equation on a square domain.  That is, find $p,\mathbf{v}:\Omega \rightarrow \mathbb{R} \times \mathbb{R}^n$ such that

$$
\begin{align*}
\nabla \cdot \mathbf{v}  &= r(x,t) \\
\frac{\partial (\rho\mathbf{v})}{\partial t} + \nabla\cdot\left(\rho\mathbf{v}\otimes\mathbf{v}\right) +  \nabla p  - \nabla \cdot \left(\mu \nabla\mathbf{v}\right) &= \rho\mathbf{f}(x,t)
\end{align*}
$$
on 
$$
\Omega = [0,1]
$$
 where $r(x,t)$ is a function of space and time only, $\mathbf{f}(x,t)$ and the density, $\rho$, and viscosity, $\mu$, are constant. 
 
To be complete and solvable we add in dirichlet boundary conditions on $p(x,t)$ and inflow boundary conditions on velocity $v(x,t)$,  (this means we must set the diffusive bc = 0 and advective bc = None to allow for the inflow bc to happen)
$$
\left.p\right|_{\partial\Omega} = p(x,t)
$$
and for $\partial\Omega^{in} = \{ x \:|\: v(x,t)\cdot n < 0\}$ we have inflow in which case we set
$$
\left.v\right|_{\partial\Omega^{in}} = v(x,t). 
$$
Finally we need initial conditions
$$
u(x,t=0) = u_0(x), \hspace{1cm} p(x,t=0) = p_0(x).
$$

In [1]:
from proteus.iproteus import * #pull in what we need for using proteus interactively
Profiling.logLevel=2
Profiling.verbose = True

Define the PDE by defining the coefficients

In [2]:
class NavierStokes1D(TransportCoefficients.TC_base):
    """
    The coefficients of the Poisson equation 
    """
    def __init__(self,rofx,fofx,rho=1.0,mu=1.0):
        TransportCoefficients.TC_base.__init__(self, 
                         nc=2, #number of components
                         variableNames=['p','v'],
                         mass = {1:{1:'linear'}}, # du/dt
                         advection = {0:{1:'linear'}, # \nabla\cdot v
                                      1:{1:'nonlinear'}}, # \nabla \cdot (v\otimes v)
                         hamiltonian = {1:{0:'linear'}}, # grad (p)
                         diffusion = {1:{1:{1:'constant'}}}, # - 1/Re * \grad v
                         potential = {1:{1:'u'}}, # define the potential for the diffusion term to be the solution itself
                         reaction  = {0:{0:'constant'}, # r(x)
                                      1:{1:'constant'}}) # f(x)
        self.rofx=rofx
        self.fofx=fofx
        self.rho=rho
        self.mu=mu
        
    def evaluate(self,t,c):
        """
        evaluate quadrature point values held in the dictionary c
        These are labelled according to the 'master equation.' For example,
         
        c[('a',0,0)] = diffusion coefficient for the 0th equation (first) with respect to the
                       0th potential (the solution itself)
                       The value at each quadrature point is a n_d x n_d tensor (n_d=number of space dimensions).
                       Usually the tensor values are stored as a flat array in compressed sparse row format to save space. 
                       By default, we assume the tensor is full though.
                       
        c[('r',0)]   = reaction term for the 0th equation. This is where we will put the source term
        """
        p = c[('u',0)]
        v = c[('u',1)] #1D - x component of velocity
        grad_p = c[('grad(u)',0)]
        #equation 0  div(f) + r = 0  (proteus  notation)  div(velocity)=r(x) (our notation)
        c[('f',0)][...,0] = v
        c[('df',0,1)][...,0] = 1.0  # d_f^0_d_u^0
        c[('r',0)][:]     = -self.rofx(c['x'][:],t)
        c[('dr',0,0)][:]   = 0.0
        #equation 1   u_t + div (u^2) + grad(p) + div(-1/Re grad(v)) + f = 0
        c[('m',1)][:] = rho*v  # d/dt ( rho * v) = d/dt (m_1)
        c[('dm',1,1)][:] = rho  # d_m^1_d_u^1
        c[('f',1)][...,0] = rho*v*v # div ( v\otimes v)
        c[('df',1,1)][...,0] = 2.0*rho*v # d_f^1_d_u^1
        c[('H',1)][:] = grad_p[...,0] #grad(p)
        c[('dH',1,0)][...,0] = 1.0 # 1
        c[('r',1)][:]     = -rho*self.fofx(c['x'][:],t)
        c[('dr',1,1)][:]  = 0.0#0
        c[('a',1,1)][...,0] = mu # -mu*\grad v
        c[('da',1,1,1)][...,0] = 0.0 # -d_(1/Re)




Setup the problem by method of manufactured solution.

We will introduce a specific example that we have implemented, but we use sympy to symbolically compute the source terms based on the navier stokes equation as defined above.  In order to change the solution, it is only necessary to update p and v below.

A specific solution to 1D navier-stokes equation:

$$
p(x,t) = 1 + \sin(2\pi (x-t))
$$
and
$$
v(x,t) = \sin(2\pi(x-t))\cos(2\pi(x-t))
$$
thus
$$
r(x,t) = \nabla\cdot v = 2\pi\cos(4\pi(x-t))
$$
and
$$
\begin{split}
f(x,t) = -2\pi\cos(4\pi(x-t)) &+ 4\pi\sin(2\pi(x-t))\cos^3(2\pi(x-t)) - 4\pi\sin^{3}(2\pi(x-t))\cos(2\pi(x-t))\\& + \frac{2\pi}{\rho}\cos(2\pi(x-t)) + \frac{8\pi^2\mu}{\rho}\sin(4\pi(x-t))
\end{split}
$$

Note that we are not divergence free here in 1D since that would require constant velocity.  The momentum equation must be written in the form  $\nabla\cdot(v\otimes v)$  instead of $v\cdot\nabla v$ since the second form utilizes the fact that $v$ is divergence free.

Define the python functions for evaluating the terms. Note that the x array for the quadrature points will be a multidimensional array. The last dimension will be 3 always though.  This is done using the sympy module and is easily modified to represent any manufactured solution desired.

In [3]:
#use numpy for evaluations
import numpy as np

# Reynolds number
rho = 1.0
mu = 1.0

# Create the manufactured solution and run through sympy 
# to create the forcing function and solutions etc
#
# Import specific sympy functions to avoid overloading 
# numpy etc functions
from sympy.utilities.lambdify import lambdify
from sympy import (symbols, 
                   simplify,
                   diff)
from sympy.functions import (sin as sy_sin,
                             cos as sy_cos)
from sympy import pi as sy_pi

# use xs and ts to represent symbolic x and t
xs,ts = symbols('x t')
# solution:
ps = 1 + sy_cos(2*sy_pi*(xs-ts))
vs = sy_sin(2*sy_pi*(xs-ts))*sy_sin(2*sy_pi*(xs-ts))

# manufacture the source terms:
rs = simplify(diff(vs,xs))
# note that we use the div(rho u\otimes u) form of convection since we are not
# divergence free
fs = simplify((diff(rho*vs,ts) + diff(rho*vs*vs,xs) + diff(ps,xs) - diff(mu*vs,xs,xs))/rho)

# use lambdify to convert from sympy to python expressions
pl = lambdify((xs, ts), ps, "numpy")
vl = lambdify((xs, ts), vs, "numpy")
rl = lambdify((xs, ts), rs, "numpy")
fl = lambdify((xs, ts), fs, "numpy")

# convert python expressions to the format we need for multidimensional x values
def ptrue(x,t): 
    return pl(x[...,0],t)

def vtrue(x,t):
    return vl(x[...,0],t)

def rofxtrue(x,t):
    return rl(x[...,0],t)

def fofxtrue(x,t):
    return fl(x[...,0],t)


Define the physical input to the model

In [4]:

physics = default_p
physics.name = "navier_stokes_1d"
physics.nd = 1; #One dimension
#spatial domain
physics.L=(1.0,1.0,1.0)

# Integrate from t=0 to t=1
physics.T = 1.0

#the object for evaluating the coefficients   
physics.coefficients=NavierStokes1D(rofxtrue,fofxtrue,rho,mu)

#this functions job is to return another function holding the Dirichlet boundary conditions 
# wherever they are set
    
eps=1.0e-8
boundaryFlags = {'left':1,'right':2,'top':3}
def getDBC_p(x,flag):
    if x[0] <= 0.0+eps:
        return lambda x,t: ptrue(x,t)
    elif x[0] >= physics.L[0]-eps:
        return lambda x,t: ptrue(x,t)
    else:
        return None
    
def getDBC_v(x,flag):
    if x[0] <= 0.0+eps:
        return lambda x,t: vtrue(x,t)
    elif x[0] >= physics.L[0]-eps:
        return lambda x,t: vtrue(x,t)
    else:
        return None
    
def getNone(x,flag):
    return None

def getZeroFlux(x,flag):
    if x[0] <= 0.0+eps:
        return lambda x,t: 0.0
    elif x[0] >= physics.L[0]-eps:
        return lambda x,t: 0.0
    else:
        return None
    
class getIBC_p:
    def __init__(self):
        self.ptrue=ptrue
        pass
    def uOfXT(self,x,t):
        return self.ptrue(x,t)

    
class getIBC_v:
    def __init__(self):
        self.vtrue=vtrue
        pass
    def uOfXT(self,x,t):
        return self.vtrue(x,t)
    
physics.initialConditions = {0:getIBC_p(),
                             1:getIBC_v()}

physics.dirichletConditions = {0:getDBC_p,#pressure Dirichlet
                               1:getDBC_v}

physics.advectiveFluxBoundaryConditions = {0:getNone}#dummy condition for non-existent  advective flux
#physics.advectiveFluxBoundaryConditions = {1:getZeroFlux}#dummy condition for non-existent  advective flux

#there are no diffusion terms in this model, but I had to 
physics.diffusiveFluxBoundaryConditions = {1:{1:getZeroFlux}}#viscous flux
physics.fluxBoundaryConditions = {0:'mixedFlow',1:'outFlow'}
#physics.fluxBoundaryConditions = {0:'setFlow',1:'setFlow'}

Define the numerics input for the problem

In [5]:
numerics=default_n
#piecewise linears
numerics.femSpaces = {0:FemTools.C0_AffineLinearOnSimplexWithNodalBasis,#pressure space
                      1:FemTools.C0_AffineQuadraticOnSimplexWithNodalBasis}#velocity space

numerics.timeIntegration = TimeIntegration.BackwardEuler
numerics.DT = 0.01
numerics.tnList = list(np.arange(0.0,physics.T+numerics.DT,numerics.DT))

#Quadrature rules for elements and element  boundaries
numerics.elementQuadrature = Quadrature.SimplexGaussQuadrature(physics.nd,5)
numerics.elementBoundaryQuadrature = Quadrature.SimplexGaussQuadrature(physics.nd-1,5)
#number of nodes in the x and y direction
numerics.nn = 101
#numerics.nny= 41
#matrix type
numerics.numericalFluxType = NumericalFlux.StrongDirichletFactory(physics.fluxBoundaryConditions)
#numerics.numericalFluxType = MixedDarcy_exterior
#numerics.numericalFluxType = NumericalFlux.Advection_DiagonalUpwind_Diffusion_IIPG_exterior
#numerics.numericalFluxType = NumericalFlux.Advection_Diagonal_average
numerics.matrix = LinearAlgebraTools.SparseMatrix
#use petsc solvers wrapped by petsc4py
#numerics.multilevelLinearSolver = LinearSolvers.KSP_petsc4py
#numerics.levelLinearSolver = LinearSolvers.KSP_petsc4py
#using petsc4py requires weak boundary condition enforcement
#can also use our internal wrapper for SuperLU
numerics.multilevelLinearSolver = LinearSolvers.LU
numerics.levelLinearSolver = LinearSolvers.LU

numerics.multilevelNonlinearSolver = NonlinearSolvers.Newton
numerics.levelNonlinearSolver = NonlinearSolvers.Newton

#linear solve rtolerance

numerics.linTolFac = 0.001
numerics.tolFac = 0.0

numerics.l_atol_res = 1.0e-5
numerics.nl_atol_res = 1.0e-5
numerics.periodicDirichletConditions=None
#
# split operator options (trivial since we're not splitting)
#
so = default_so
so.name = physics.name
so.sList=[default_s]
so.tnList = numerics.tnList

Construct a NumericalSolution object and solve the problem

In [8]:
ns = NumericalSolution.NS_base(so,[physics],[numerics],so.sList,opts)
ns.calculateSolution('run1')

[      96] Initializing NumericalSolution for navier_stokes_1d
 System includes: 
navier_stokes_1d

[      96] Setting Archiver(s)
[      96] Setting up MultilevelMesh
[      96] Building one multilevel mesh for all models
[      96] Generating mesh for navier_stokes_1d
[      96] Partitioning mesh among 1 processors using partitioningType = 0
[      96] Number of Subdomain Elements Owned= 100
[      96] Number of Subdomain Elements = 100
[      96] Number of Subdomain Nodes Owned= 101
[      96] Number of Subdomain Nodes = 101
[      96] Number of Subdomain elementBoundaries Owned= 101
[      96] Number of Subdomain elementBoundaries = 101
[      96] Number of Subdomain Edges Owned= 100
[      96] Number of Subdomain Edges = 100
[      96] Finished partitioning
[      96] *** Global ***
Number of edges : 100
Number of nodes : 101

*** Local ***
Number of edges : 100
Number of nodes : 101

[      96] Setting up MultilevelTransport for navier_stokes_1d
[      96] Allocating phi
[      9

IndexError: list index out of range

Plot the solution with matplotlib. Only works for $C^0 P^1$ right now

In [33]:
# extract computed solutions and mesh

#arrays for using matplotlib's unstructured plotting interface
x = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:,0]
#y = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:,1]
#triangles = ns.modelList[0].levelModelList[-1].mesh.elementNodesArray

# grab the last time step solution and save the components as p and v
p = ns.modelList[0].levelModelList[-1].u[0].dof
v = ns.modelList[0].levelModelList[-1].u[1].dof


# note that thedata is saved in xml and hf5 files at each time period in the numerics.tnList
# so we must extract it and can make an animation from that data
import tables
h5 = tables.openFile("navier_stokes_1d0.h5","r")
print h5
#mesh nodes, elements, and solution fields at step 0
nodeArray = h5.getNode("/nodesSpatial_Domain0")[:]
elementNodesArray = h5.getNode("/elementsSpatial_Domain0")[:]
p =  h5.getNode("/p0")[:]
# u =  h5.getNode("/u0")[:]
v=  h5.getNode("/v0")[:]

#step 1
p =  h5.getNode("/p%i" %3)[:]
ptest =  h5.getNode("/p3")[:]
print abs(p-ptest)
# u =  h5.getNode("/u1")[:]
v=  h5.getNode("/v1")[:]



navier_stokes_1d0.h5 (File) 'navier_stokes_1d Data'
Last modif.: 'Mon Jun  1 15:51:24 2015'
Object Tree: 
/ (RootGroup) 'navier_stokes_1d Data'
/cellMapL2GSpatial_Domain0 (Array(100,)) ''
/cellMapL2GSpatial_Domain1 (Array(100,)) ''
/cellMapL2GSpatial_Domain10 (Array(100,)) ''
/cellMapL2GSpatial_Domain100 (Array(100,)) ''
/cellMapL2GSpatial_Domain11 (Array(100,)) ''
/cellMapL2GSpatial_Domain12 (Array(100,)) ''
/cellMapL2GSpatial_Domain13 (Array(100,)) ''
/cellMapL2GSpatial_Domain14 (Array(100,)) ''
/cellMapL2GSpatial_Domain15 (Array(100,)) ''
/cellMapL2GSpatial_Domain16 (Array(100,)) ''
/cellMapL2GSpatial_Domain17 (Array(100,)) ''
/cellMapL2GSpatial_Domain18 (Array(100,)) ''
/cellMapL2GSpatial_Domain19 (Array(100,)) ''
/cellMapL2GSpatial_Domain2 (Array(100,)) ''
/cellMapL2GSpatial_Domain20 (Array(100,)) ''
/cellMapL2GSpatial_Domain21 (Array(100,)) ''
/cellMapL2GSpatial_Domain22 (Array(100,)) ''
/cellMapL2GSpatial_Domain23 (Array(100,)) ''
/cellMapL2GSpatial_Domain24 (Array(100,)) ''
/ce

In [ ]:
%matplotlib inline

Compare to the exact solution, evaluated at the degrees of freedom. Again, only works for C0P1 elements

Compare to the exact solution, evaluated at the degrees of freedom. Again, only works for C0P1 elements

In [ ]:
# extract exact solutions at time T
p_true = ptrue(ns.modelList[0].levelModelList[-1].mesh.nodeArray, physics.T)
v_true = vtrue(ns.modelList[0].levelModelList[-1].mesh.nodeArray, physics.T)
r_true = rofxtrue(ns.modelList[0].levelModelList[-1].mesh.nodeArray, physics.T)

# plot computed solutions
import matplotlib.pyplot as plt

# use matplotlib.rc to change titles and labels to process latex code and change fonts
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

# Plotting Pressure

# Set plotting features including legend and formatting colors and shading
fig, (ax1,ax2) = plt.subplots(2,1)
fig.tight_layout() # spread out the plots so that they don't overlap
ax1.plot(x,p,'rx-',label='$\widehat{p}(%s)$' %physics.T)
ax1.plot(x,p_true,'b-',label='$p(%s)$' %physics.T)
ax1.set_xlabel('x')
ax1.set_ylabel('pressure')
ax1_legend = ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., shadow=True)
ax1_frame = ax1_legend.get_frame()
ax1_frame.set_facecolor('0.90')
# Set the fontsize
for label in ax1_legend.get_texts():
    label.set_fontsize('large')
for label in ax1_legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width

    
# plotting Velocity

useTH=True
if not useTH:
    ax2.plot(x, v[:numerics.nn],'r+-',label='$\widehat{v}(%s)$' %physics.T)
else:
    # This is hard coded for plotting P^2 elements for velocity
    xm = np.array(0.5*(x[:-1]+x[1:]))
    xq=[];
    vq=[];
    pq=[];
    for i in range(numerics.nn-1):
        xq.append(x[i])
        vq.append(v[i])
        #uq.append(u[i])
        xq.append(xm[i])
        vq.append(v[numerics.nn+i])
        #uq.append(u[numerics.nn+i])
    xq.append(x[-1])
    vq.append(v[-1])
    #uq.append(u[-1])
    #ax2.plot(xq,uq)
    ax2.plot(xq,vq,'r+-',label='$\widehat{v}(%s)$' %physics.T)

# add formatting and legend to ax2 plot on the outside right of plot
ax2.set_xlabel('x')
ax2.set_ylabel('velocity')
ax2.plot(x, v_true, 'b-',label='$v(%s)$' %physics.T)
ax2_legend = ax2.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., shadow=True)
ax2_frame = ax2_legend.get_frame()
ax2_frame.set_facecolor('0.90')
# Set the fontsize
for label in ax2_legend.get_texts():
    label.set_fontsize('large')
for label in ax2_legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
plt.show()

Compute the discrete maximum norm and $\ell_2$ norms for velocity and pressure.

In [ ]:

# pressure error and plotting
print 'Pressure P^1 elements:  Number of Nodes: %s' %(x.shape[0])
fig, (ax1,ax2) = plt.subplots(2,1)
fig.tight_layout() # spread out the plots so that they don't overlap
err_p = p - p_true
print "Maximal error of pressure(T) on gridpoints           = %s" % max(abs(err_p))
ax1.plot(x,abs(err_p),label='$|p-\hat{p}|(t=%s)$' %physics.T)
ax1.set_xlabel('x')
ax1.set_ylabel('$e_p$')
ax1_legend = ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., shadow=True)
ax1_frame = ax1_legend.get_frame()
ax1_frame.set_facecolor('0.90')
# Set the fontsize
for label in ax1_legend.get_texts():
    label.set_fontsize('large')
for label in ax1_legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
    
# finish calculating the ell2 norm for pressure
err_p *= err_p
err_p *= physics.L[0]*physics.L[1]/(numerics.nn)
L2approx_p = np.sqrt(err_p.sum())
print "Trapezoidal rule approx to \ell_2 error of pressure(T) = %s\n" % L2approx_p


# Velocity error and plotting
print 'Velocity P^2 elements:  Number of Nodes: %s,  Midpoints: %s' %(x.shape[0],xm.shape[0])

# note that the x values for $v_hat_at_T \in\mathbb{P}^2(\mathcal{T})$  are given nodes then midpoints [x,xm]
xvals = np.concatenate((x,xm))
v_true = 0.0*v
for i,val in enumerate(xvals):
    v_true[i] = vtrue(np.array([val]),physics.T)
err_v = v - v_true
# now v_hat_at_T and vtrue and err_v have the same order: nodes then midpoints for P^2 elements

# sort the x values and reorder for plotting
ind = np.argsort(xvals)
xvals_sort = xvals[ind]
err_v_sort = err_v[ind]

print "Maximal error of velocity on gridpoints                        is %s" % max(abs(err_v))
ax2.plot(xvals_sort,err_v_sort,label='$|\hat{v}-v|(t=%s)$' %physics.T)
ax2.set_xlabel('x')
ax2.set_ylabel('$e_v$')
ax2_legend = ax2.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., shadow=True)
ax2_frame = ax2_legend.get_frame()
ax2_frame.set_facecolor('0.90')
# Set the fontsize
for label in ax2_legend.get_texts():
    label.set_fontsize('large')
for label in ax2_legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width

err_v *= err_v
err_v *= physics.L[0]*physics.L[1]/(numerics.nn)
L2approx_v = np.sqrt(err_v.sum())
print "Trapezoidal rule approx to \ell_2 error of velocity for nn=%s is %s" % (numerics.nn, L2approx_v)

# adjust the plots to be up to date
plt.show()